#### Install Dependencies

In [ ]:
!sudo apt-get install jags
!pip install pyjags

# Bayesian Parameter Estimation for Factor Contributions

The following is a simplified template for using JAGS for analyses

In [ ]:
import pandas as pd
import numpy as np
import pyjags
import os

In [ ]:
DATA_PATH = 'content'
INPUT_FILE = 'ceda-results.csv'

### Read in data

In [ ]:
df = pd.read_csv(os.path.join(DATA_PATH,INPUT_FILE))

In [ ]:
# add any additional processing steps here

In [ ]:
df.head()

## Model Definition

### LME Model

The following is an example JAGS Script. It replicates the logic inherent to LME Regression Analysis (with no interactions) in a JAGS framework.

The following script is basic, and unlikely to fit all your analysis needs. You'll likely need t update it to remove and or include additional variables based on your specific hypotheses. Use it as a guideline for how to write an appropriate JAGS script.

In [ ]:
model = """
model{
    asigma ~ dunif(.001, 100)
    rsigma ~ dunif(.001, 100)
    csigma ~ dunif(.001, 100)
    nxsigma ~ dunif(.001, 100)
    nysigma ~ dunif(.001, 100)
    
    intercept ~ dunif(-1000, 1000)
    amu ~ dunif(-1000, 1000)
    rmu ~ dunif(-1000, 1000)
    cmu ~ dunif(-1000, 1000)
    nxmu ~ dunif(-1000, 1000)
    nymu ~ dunif(-1000, 1000)
    
    nx_beta ~ dnorm(nxmu, nxsigma)
    ny_beta ~ dnorm(nymu, nysigma)
    
    likes_beta ~ dunif(-1000, 1000)
    
    for (a in 1:AUTHORS){
        amu_[a] ~ dnorm(amu, asigma)
        asigma_[a] ~ dunif(.001,100)
        authors_beta[a] ~ dnorm(amu_[a], asigma_[a])
    }
    
    for (r in 1:REPLIES){
        rmu_[r] ~ dnorm(rmu, rsigma)
        rsigma_[r] ~ dunif(.001,100)
        repliers_beta[r] ~ dnorm(rmu_[r], rsigma_[r])
    }
    
    for (c in 1:COMMENTS){
        cmu_[c] ~ dnorm(cmu, csigma)
        csigma_[c] ~ dunif(.001,100)
        
        comments_beta[c] ~ dnorm(cmu_[c], csigma_[c])
        
        gamma[c] ~ dunif(.001, 100)
    }
    
    for (i in 1:ROWS){
        mu_row[i] <- intercept + authors_beta[authors[i]] + repliers_beta[repliers[i]] + comments_beta[comments[i]] + (likes_beta * likes[i]) + (nx_beta * nx[i]) + (ny_beta * ny[i])
        H[i] ~ dnorm(mu_row[i], TEMP)
    }
}
"""

**[ON ADDING INTERACTION TERMS]**

Somewhat obviously, the above is completely hand crafted. That means that if one wants to add in interaction terms, there's some extra work involved. That work includes (1) creating Z-Scores for each interacting variable, and (2) taking the product of those Z-Scores. We can then (3) calculated a $\beta$ value for the interaction of those terms.

Make sure that you include inputs for each variable that you are feeding to the model from the data, as indicated in the script you wrote.

In [ ]:
data = {
    'authors': df['authors'].values,
    'repliers': df['repliers'].values,
    'comments': df['comment_id'].values,
    'likes': df['likes'].values,
    'nx': df['nx'].values,
    'ny': df['ny'].values,
    
    'H': df['Hxy'].values,
    
    'AUTHORS': df['authors'].nunique(),
    'REPLIERS': df['repliers'].nunique(),
    'COMMENTS': df['comment_id'].nunique(),
    'LIKES': df['likes'].nunique(),
    'TEMP': 1, # how far off from the correct value of H you are allowing the model to estimate.
                # higher values will increase model performance, but decrease model certainty
                # for any specific parameter values.
    'ROWS': len(df)
}

In [ ]:
n_samples = 3000
chains = 1
warm_up = 1000

jags_model = pyjags.Model(
    model,
    data=data,
    chains=chains,
    adapt=warm_up, 
    progress_bar=True
)

samples = jags_model.sample(n_samples,vars=['intercept', 'amu', 'rmu', 'cmu', 'likes_beta'])

saving outputs

In [ ]:
for param,vals in samples.items():
    np.save(
        os.path.join(DATA_PATH, param+'.npy'), 
        vals
    )